In [ ]:
%pip install fastapi nest-asyncio pyngrok uvicorn jinja2 numpy scikit-learn voila requests


In [ ]:
import json
import requests
from starlette.responses import FileResponse
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import nest_asyncio
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from fastapi import FastAPI, Request
import pickle
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [ ]:
class IrisSpecies(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float


In [ ]:
app = FastAPI()


In [ ]:
@app.get('/')
async def sample():
    loaded_X = pickle.load(open('X_train.pkl', 'rb'))
    x0 = np.random.normal(loaded_X[:, 0].mean(), loaded_X[:, 0].std(), 1)
    x1 = np.random.normal(loaded_X[:, 1].mean(), loaded_X[:, 1].std(), 1)
    x2 = np.random.normal(loaded_X[:, 2].mean(), loaded_X[:, 2].std(), 1)
    x3 = np.random.normal(loaded_X[:, 3].mean(), loaded_X[:, 3].std(), 1)

    return {
        'sepal_length': x0[0],
        'sepal_width': x1[0],
        'petal_length': x2[0],
        'petal_width': x3[0]
    }


In [ ]:
@app.post('/')
async def predict(iris: IrisSpecies):
    data = iris.dict()
    loaded_model = pickle.load(open('model.pkl', 'rb'))
    data_in = [[data['sepal_length'], data['sepal_width'],
                data['petal_length'], data['petal_width']]]
    prediction = loaded_model.predict(data_in)
    probability = loaded_model.predict_proba(data_in).max()
    iris_type = ['Iris Setosa', 'Iris Virginica', 'Iris Versicolor']

    return {
        'prediction': int(prediction[0]),
        'iris_specie': iris_type[int(prediction[0])-1],
        'probability': probability
    }


In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)


In [ ]:
# ngrok.kill()

In [ ]:
# taskkill /f /im ngrok.exe

In [ ]:
# %pip freeze > requirements.txt


In [ ]:
# version = !python - -version
# version = version[0].lower().replace(" ", "-")
# !echo $version > runtime.txt


In [ ]:
# %echo web: voila - -port =$PORT - -no-browser api-colab.ipynb > Procfile
